# Analysis

This notebook is NEW CONTINUING: I am piping in data objects created from other notebooks and analyzing them.

I will be combining datasets here and analyzing them based on my research questions.

## Loading in the data

In [1]:
# loading packages
import numpy as np
import pandas as pd
import nltk
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
f1 = open('characters_update_df.pkl', 'rb')
characters_df = pickle.load(f1)
f1.close()

# f2 = open('conversations_df.pkl', 'rb')
# conversations_df = pickle.load(f2)
# f2.close()

f3 = open('movies_df.pkl', 'rb')
movies_df = pickle.load(f3)
f3.close

f4 = open('utterances_df.pkl', 'rb')
utterances_df = pickle.load(f4)
f4.close

<function BufferedReader.close>

In [3]:
# should i drop credit_position in initial notebook?
characters_df.head().drop('credit_position', axis=1, inplace=True)

<ipython-input-3-34c08b194ad6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  characters_df.head().drop('credit_position', axis=1, inplace=True)


In [4]:
# conversations_df.head()

In [5]:
movies_df.head()

,movie_title,movie_year,genres,movie_decade
0,10 things i hate about you,1999,"['comedy', 'romance']",1990
1,1492: conquest of paradise,1992,"['adventure', 'biography', 'drama', 'history']",1990
2,15 minutes,2001,"['action', 'crime', 'drama', 'thriller']",2000
3,2001: a space odyssey,1968,"['adventure', 'mystery', 'sci-fi']",1960
4,48 hrs.,1982,"['action', 'comedy', 'crime', 'drama', 'thrill...",1980


In [6]:
utterances_df.head()

,line_ID,character_ID,movie_ID,character_name,utterance,sents,tokens,pos_tag,sent_count,token_count,avg_sent_length
0,L1045,u0,m0,BIANCA,They do not!,[They do not!],"[They, do, not, !]","[(They, PRP), (do, VBP), (not, RB), (!, .)]",1,4,4.0
1,L1044,u2,m0,CAMERON,They do to!,[They do to!],"[They, do, to, !]","[(They, PRP), (do, VBP), (to, TO), (!, .)]",1,4,4.0
2,L985,u0,m0,BIANCA,I hope so.,[I hope so.],"[I, hope, so, .]","[(I, PRP), (hope, VBP), (so, RB), (., .)]",1,4,4.0
3,L984,u2,m0,CAMERON,She okay?,[She okay?],"[She, okay, ?]","[(She, PRP), (okay, PRP), (?, .)]",1,3,3.0
4,L925,u0,m0,BIANCA,Let's go.,[Let's go.],"[Let, 's, go, .]","[(Let, VB), ('s, POS), (go, VB), (., .)]",1,4,4.0


## Compiling the Data

### Linguistic Analysis

In [7]:
# because I am looking at discourse, I will use the utterances data frame to load other data into

# adding gender
discourse_df = pd.merge(utterances_df, characters_df)

# adding movie year and decade
discourse_df = pd.merge(discourse_df, movies_df)

In [8]:
# there are 267 utterances that are empty, I will remove them
discourse_df = discourse_df[discourse_df.token_count != 0]

In [9]:
discourse_df.head()

,line_ID,character_ID,movie_ID,character_name,utterance,sents,tokens,pos_tag,sent_count,token_count,avg_sent_length,movie_title,gender,credit_position,movie_year,genres,movie_decade
0,L1045,u0,m0,BIANCA,They do not!,[They do not!],"[They, do, not, !]","[(They, PRP), (do, VBP), (not, RB), (!, .)]",1,4,4.000000,10 things i hate about you,F,4,1999,"['comedy', 'romance']",1990
1,L985,u0,m0,BIANCA,I hope so.,[I hope so.],"[I, hope, so, .]","[(I, PRP), (hope, VBP), (so, RB), (., .)]",1,4,4.000000,10 things i hate about you,F,4,1999,"['comedy', 'romance']",1990
2,L925,u0,m0,BIANCA,Let's go.,[Let's go.],"[Let, 's, go, .]","[(Let, VB), ('s, POS), (go, VB), (., .)]",1,4,4.000000,10 things i hate about you,F,4,1999,"['comedy', 'romance']",1990
3,L872,u0,m0,BIANCA,Okay -- you're gonna need to learn how to lie.,[Okay -- you're gonna need to learn how to lie.],"[Okay, --, you, 're, gon, na, need, to, learn,...","[(Okay, NNP), (--, :), (you, PRP), ('re, VBP),...",1,13,13.000000,10 things i hate about you,F,4,1999,"['comedy', 'romance']",1990
4,L870,u0,m0,BIANCA,I'm kidding. You know how sometimes you just ...,"[I'm kidding., You know how sometimes you just...","[I, 'm, kidding, ., You, know, how, sometimes,...","[(I, PRP), ('m, VBP), (kidding, VBG), (., .), ...",3,25,8.333333,10 things i hate about you,F,4,1999,"['comedy', 'romance']",1990


In [10]:
discourse_df.dropna().describe()

,sent_count,token_count,avg_sent_length,movie_year,movie_decade
count,304446.000000,304446.000000,304446.000000,304446.000000,304446.000000
mean,1.695335,13.734593,7.855335,1988.107372,1983.436110
std,1.252310,14.712174,5.155574,17.140874,17.203622
min,1.000000,1.000000,1.000000,1927.000000,1920.000000
25%,1.000000,5.000000,4.500000,1984.000000,1980.000000
50%,1.000000,9.000000,7.000000,1995.000000,1990.000000
75%,2.000000,17.000000,10.000000,1999.000000,1990.000000
max,45.000000,684.000000,122.000000,2010.000000,2010.000000


There is a wide range on the token counts, from 1 to 684. There is also a wide range on sentence counts, from 1 to 45. I need to decide if I will include the max values in my analysis to follow. For now they will remain in.

#### Tokens

I am looking at token counts to see if there is a difference between how turns are by gender.

In [12]:
discourse_df.groupby('gender').token_count.mean()

gender
A          13.401938
F          13.109015
M          13.892254
unknown    14.542629
Name: token_count, dtype: float64

Only 0.7 difference between average token counts of male and female characters. Characters with unknown gender markers have the longest turns at 14.49. If I am able to get more of these identified that will be benficial to my analysis.

In [13]:
discourse_df.groupby('gender').token_count.sum()

gender
A            81591
F          1077312
M          2589794
unknown     432745
Name: token_count, dtype: int64

Raw count doesn't help for comparison across categories, but by looking at this we can see that there are more male characters because the average token count above does not differ as much as the raw counts.

In [14]:
discourse_df.groupby('gender').token_count.min()

gender
A          1
F          1
M          1
unknown    1
Name: token_count, dtype: int64

Characters across all gender markers have utterances that are only one token long. I expect these will be interjections of some kind.

In [15]:
discourse_df[discourse_df.token_count==1]

,line_ID,character_ID,movie_ID,character_name,utterance,sents,tokens,pos_tag,sent_count,token_count,avg_sent_length,movie_title,gender,credit_position,movie_year,genres,movie_decade
10,L698,u0,m0,BIANCA,But,[But],[But],"[(But, CC)]",1,1,1.0,10 things i hate about you,F,4,1999,"['comedy', 'romance']",1990
13,L663,u0,m0,BIANCA,Tons,[Tons],[Tons],"[(Tons, NNS)]",1,1,1.0,10 things i hate about you,F,4,1999,"['comedy', 'romance']",1990
61,L904,u0,m0,BIANCA,But,[But],[But],"[(But, CC)]",1,1,1.0,10 things i hate about you,F,4,1999,"['comedy', 'romance']",1990
100,L924,u2,m0,CAMERON,Wow,[Wow],[Wow],"[(Wow, NN)]",1,1,1.0,10 things i hate about you,M,3,1999,"['comedy', 'romance']",1990
101,L871,u2,m0,CAMERON,No,[No],[No],"[(No, DT)]",1,1,1.0,10 things i hate about you,M,3,1999,"['comedy', 'romance']",1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298537,L646110,u8846,m600,CHINA,Yeah,[Yeah],[Yeah],"[(Yeah, NN)]",1,1,1.0,waxwork,unknown,?,1988,"['comedy', 'fantasy', 'horror']",1980
298598,L646365,u8851,m600,MARK,China,[China],[China],"[(China, NNP)]",1,1,1.0,waxwork,M,?,1988,"['comedy', 'fantasy', 'horror']",1980
298631,L646309,u8853,m600,MR LINCOLN,Hi,[Hi],[Hi],"[(Hi, NN)]",1,1,1.0,waxwork,M,?,1988,"['comedy', 'fantasy', 'horror']",1980
302290,L659811,u8950,m610,DOROTHY,Oh-oh-,[Oh-oh-],[Oh-oh-],"[(Oh-oh-, NN)]",1,1,1.0,the wizard of oz,F,1,1939,"['adventure', 'family', 'fantasy', 'musical']",1930


At quick glance, some interjections and some answers.

In [16]:
discourse_df.groupby('gender').token_count.max()

gender
A          225
F          368
M          684
unknown    337
Name: token_count, dtype: int64

In [17]:
discourse_df[discourse_df.token_count==684]
# this is the longest token count per utterance
# seems like it may be a narration intro...it may be removed from analysis

,line_ID,character_ID,movie_ID,character_name,utterance,sents,tokens,pos_tag,sent_count,token_count,avg_sent_length,movie_title,gender,credit_position,movie_year,genres,movie_decade
182764,L217873,u5313,m352,PARRY,Then let's begin with the story itself. It's a...,"[Then let's begin with the story itself., It's...","[Then, let, 's, begin, with, the, story, itsel...","[(Then, RB), (let, NN), ('s, POS), (begin, VB)...",26,684,26.307692,the fisher king,M,19,1991,"['comedy', 'drama', 'romance']",1990


At the total corpus level there are not any huge differences between gender at the token level. I will add factor in movie decade to see if any differences can be detected across time. 

#### Token and Movie Decade

In [32]:
discourse_df.groupby(['movie_decade','gender']).token_count.mean()

movie_decade  gender 
1920          A          10.882353
              F           6.339623
              M          10.054054
              unknown     8.367647
1930          A           9.478261
              F          13.981007
              M          15.448819
              unknown    15.005722
1940          A          16.761062
              F          15.086478
              M          15.280879
              unknown    14.686606
1950          A          13.985915
              F          14.323550
              M          13.958992
              unknown    13.604294
1960          A          14.000000
              F          12.061100
              M          14.073752
              unknown    14.868297
1970          A          13.052326
              F          12.912455
              M          13.853650
              unknown    14.946744
1980          A          13.240175
              F          12.190641
              M          13.105019
              unknown    14.06903

In [33]:
discourse_df.groupby(['movie_decade','gender']).token_count.min()
# looks like not all movies have as short of utterances

movie_decade  gender 
1920          A          2
              F          2
              M          2
              unknown    2
1930          A          2
              F          1
              M          1
              unknown    1
1940          A          2
              F          1
              M          1
              unknown    1
1950          A          2
              F          1
              M          2
              unknown    2
1960          A          2
              F          1
              M          2
              unknown    1
1970          A          1
              F          1
              M          1
              unknown    1
1980          A          1
              F          1
              M          1
              unknown    1
1990          A          2
              F          1
              M          1
              unknown    1
2000          A          2
              F          1
              M          1
              unknown    2
2010  

In [34]:
discourse_df.groupby(['movie_decade','gender']).token_count.max()

movie_decade  gender 
1920          A           33
              F           20
              M          165
              unknown     39
1930          A           54
              F          160
              M          227
              unknown    337
1940          A          146
              F          229
              M          358
              unknown    118
1950          A           77
              F          317
              M          247
              unknown    120
1960          A           63
              F          238
              M          366
              unknown    126
1970          A           93
              F          239
              M          253
              unknown    208
1980          A          197
              F          368
              M          266
              unknown    173
1990          A          147
              F          243
              M          684
              unknown    243
2000          A          225
              F      

#### Sentences

In [18]:
discourse_df.groupby('gender').sent_count.mean()

gender
A          1.666557
F          1.654945
M          1.707730
unknown    1.735121
Name: sent_count, dtype: float64

The average utterance is less than two sentences long. There is not much difference across the categories. On average, male utterances have slightly more sentences. Again, unknown gender has the most (but barely) average sentences per utterance.

In [19]:
discourse_df.groupby('gender').sent_count.sum()
# more sentences for male characters

gender
A           10146
F          136005
M          318355
unknown     51632
Name: sent_count, dtype: int64

This is another view of what we saw above: there are more male characters, which is why the raw sentence count is higher but not the average sentence count.

In [20]:
discourse_df.groupby('gender').avg_sent_length.mean()
# average sentence length across all genders does not seem to be too wide of a spread

gender
A          7.894995
F          7.669146
M          7.876876
unknown    8.226484
Name: avg_sent_length, dtype: float64

For each gender, sentences have on average around 8 words, although female characters have the shortest sentences at 7.67.

In [21]:
discourse_df.groupby('gender').sent_count.max()
# the longest sentences 

gender
A          18
F          33
M          45
unknown    24
Name: sent_count, dtype: int64

In [22]:
discourse_df[discourse_df.sent_count==45]

,line_ID,character_ID,movie_ID,character_name,utterance,sents,tokens,pos_tag,sent_count,token_count,avg_sent_length,movie_title,gender,credit_position,movie_year,genres,movie_decade
205014,L311961,u6045,m402,GEORGE,"Just a minute  just a minute. Now, hold on, ...","[Just a minute  just a minute., Now, hold on...","[Just, a, minute, , just, a, minute, ., Now,...","[(Just, RB), (a, DT), (minute, JJ), (, NN), ...",45,358,7.955556,it's a wonderful life,M,1,1946,"['drama', 'fantasy', 'romance']",1940


Both the longest utterance by sentence and longest utterance by token count are from male characters.

How does the sentence level information change by decade?

#### Sentences and Movie Decade

In [35]:
discourse_df.groupby(['movie_decade','gender']).sent_count.mean()

movie_decade  gender 
1920          A          1.352941
              F          1.169811
              M          1.432432
              unknown    1.367647
1930          A          1.623188
              F          1.700855
              M          1.761319
              unknown    1.814020
1940          A          1.849558
              F          1.721698
              M          1.773227
              unknown    1.730819
1950          A          1.605634
              F          1.658980
              M          1.622535
              unknown    1.643149
1960          A          1.354839
              F          1.511609
              M          1.620451
              unknown    1.635936
1970          A          1.665698
              F          1.591336
              M          1.635119
              unknown    1.657056
1980          A          1.676856
              F          1.625088
              M          1.683395
              unknown    1.745981
1990          A          1

In [37]:
discourse_df.groupby(['movie_decade','gender']).sent_count.min()

movie_decade  gender 
1920          A          1
              F          1
              M          1
              unknown    1
1930          A          1
              F          1
              M          1
              unknown    1
1940          A          1
              F          1
              M          1
              unknown    1
1950          A          1
              F          1
              M          1
              unknown    1
1960          A          1
              F          1
              M          1
              unknown    1
1970          A          1
              F          1
              M          1
              unknown    1
1980          A          1
              F          1
              M          1
              unknown    1
1990          A          1
              F          1
              M          1
              unknown    1
2000          A          1
              F          1
              M          1
              unknown    1
2010  

In [36]:
discourse_df.groupby(['movie_decade','gender']).sent_count.max()

movie_decade  gender 
1920          A           3
              F           2
              M          14
              unknown     4
1930          A           6
              F          14
              M          20
              unknown    18
1940          A           7
              F          19
              M          45
              unknown    10
1950          A           6
              F          26
              M          16
              unknown    12
1960          A           3
              F          18
              M          25
              unknown    17
1970          A          13
              F          18
              M          23
              unknown    15
1980          A          18
              F          33
              M          22
              unknown    17
1990          A          14
              F          20
              M          39
              unknown    24
2000          A          15
              F          21
              M          3

#### POS

I will ignore noun and verbs, but will look for other POS to see if usage differs by gender.

The parts of speech I will look at are the following:
* RB (adverb)
* RBR (adverb comparative)
* RBS (adverb superlative)
* JJ (adjective)
* JJR (comparative adjective)
* JJS (superlative adjective)
* CC (conjunction)
* UH (interjection)

#### Lexical Items

I will look to see if these hedging words appear more in one specific gender's speech or not:
* I think
* I guess
* Maybe
* Might
* Possibly
* Perhaps

### Gender/Movie Analysis

In [23]:
# it may be interesting to see character/gender information by year/decade

gender_df = pd.merge(characters_df, movies_df)

In [24]:
gender_df.head()

,character_ID,character_name,movie_ID,movie_title,gender,credit_position,movie_year,genres,movie_decade
0,u1,BRUCE,m0,10 things i hate about you,M,?,1999,"['comedy', 'romance']",1990
1,u3,CHASTITY,m0,10 things i hate about you,F,?,1999,"['comedy', 'romance']",1990
2,u8,MISS PERKY,m0,10 things i hate about you,F,?,1999,"['comedy', 'romance']",1990
3,u10,SHARON,m0,10 things i hate about you,F,?,1999,"['comedy', 'romance']",1990
4,u0,BIANCA,m0,10 things i hate about you,F,4,1999,"['comedy', 'romance']",1990


In [30]:
gender_df.groupby(['movie_decade', 'gender']).count()

character_ID  character_name  movie_ID  movie_title  \
movie_decade gender                                                         
1920         A                   2               2         2            2   
             F                   4               4         4            4   
             M                   3               3         3            3   
             unknown             4               4         4            4   
1930         A                  10              10        10           10   
             F                  55              55        55           55   
             M                 117             117       117          117   
             unknown            83              83        83           83   
1940         A                  16              16        16           16   
             F                  62              62        62           62   
             M                 103             103       103          103   
             unknown            64              64        64           64   
1950         A                  10              10        10           10   
             F                  56              56        56           56   
             M                  89              89        89           89   
             unknown            80              80        80           80   
1960         A                   6               6         6            6   
             F                  59              59        59           59   
             M                  97              97        97           97   
             unknown           109             109       109          109   
1970         A                  48              48        48           48   
             F                 161             161       161          161   
             M                 328             328       328          328   
             unknown           312             312       312          312   
1980         A                  94              94        94           94   
             F                 340             340       340          340   
             M                 601             601       601          601   
             unknown           488             488       488          488   
1990         A                 236             236       236          236   
             F                 801             801       801          801   
             M                1482            1482      1482         1482   
             unknown          1065            1065      1065         1065   
2000         A                 126             126       126          126   
             F                 500             500       500          500   
             M                 773             773       773          773   
             unknown           633             633       633          633   
2010         A                   3               3         3            3   
             F                   4               4         4            4   
             M                   6               6         6            6   
             unknown             5               5         5            5   

                      credit_position  movie_year  genres  
movie_decade gender                                        
1920         A                      2           2       2  
             F                      4           4       4  
             M                      3           3       3  
             unknown                4           4       4  
1930         A                     10          10      10  
             F                     55          55      55  
             M                    117         117     117  
             unknown               83          83      83  
1940         A                     16          16      16  
             F                     62          62      62  
             M                    103         103     103  
       

In [27]:
gender_df[gender_df.movie_decade==1920]

,character_ID,character_name,movie_ID,movie_title,gender,credit_position,movie_year,genres,movie_decade
2068,u2068,CHRISTOPH,m134,metropolis,M,?,1927,"['adventure', 'drama', 'sci-fi']",1920
2069,u2069,DEX,m134,metropolis,unknown,?,1927,"['adventure', 'drama', 'sci-fi']",1920
2070,u2070,DIGITAL GENA,m134,metropolis,unknown,?,1927,"['adventure', 'drama', 'sci-fi']",1920
2071,u2071,EMMA,m134,metropolis,F,?,1927,"['adventure', 'drama', 'sci-fi']",1920
2072,u2072,GENA,m134,metropolis,F,?,1927,"['adventure', 'drama', 'sci-fi']",1920
2073,u2073,HALDEN,m134,metropolis,unknown,?,1927,"['adventure', 'drama', 'sci-fi']",1920
2074,u2075,KID,m134,metropolis,A,?,1927,"['adventure', 'drama', 'sci-fi']",1920
2075,u2076,LAWYER,m134,metropolis,A,?,1927,"['adventure', 'drama', 'sci-fi']",1920
2076,u2077,SYNTHETIC VOICE,m134,metropolis,unknown,?,1927,"['adventure', 'drama', 'sci-fi']",1920
2077,u2078,TANNER,m134,metropolis,M,?,1927,"['adventure', 'drama', 'sci-fi']",1920


Aside from the 1920s, which only had two movies, all other decades have more male characters than female characters.